In [ ]:
# from __future__ import unicode_literals
import os
from bs4 import BeautifulSoup
import requests
import re
import xlrd
from openpyxl import Workbook
import time
from tqdm import tqdm
from lxml import etree
import json

def date_time():
    dt=time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()) )
    return dt

def main_spider(year):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
    #payload = {'spm':'a213w.3064813.a214dqe.22','province': '%B9%E3%CE%F7'}

    
    req = requests.get('https://jieqi.911cha.com/'+str(year)+'.html',timeout=5,headers=headers)
    soup = BeautifulSoup(req.content, "lxml")

    html=etree.HTML(req.text)
    _jq_names= html.xpath('//div[@class="gclear jq mb"]/a/span/text()')
    _jq_times=html.xpath('//div[@class="gclear jq mb"]/a/text()')

    jq_names=[]   
    
#     print(len(_jq_times))
    if len(_jq_times)==48:
        lst_j=[]
        lst_o=[]
        for i in range(0,len(_jq_times)):
            if i%2==0:
                lst_o.append(_jq_times[i])
            else:
                lst_j.append(_jq_times[i])

        lst=[]
        for i in range(0,len(lst_j)):
            s=lst_o[i]+" "+lst_j[i]
            lst.append(s)

        for i in range(0,len(lst_j)):
            s=lst_o[i]+" "+lst_j[i]
            lst.append(s)
    elif len(_jq_times)==24:  #1900年前，该网站没有农历
        lst=_jq_times        
        
    else: #2034年的雨水没有农历会导致报错
        
        _jq_times.insert(3,'农历 十二月(大)三十')
        lst=_jq_times 
#         print(lst)
#         print('其他错误')

        
#     print(lst)
    
    lst_jq={}
    name_jq=["立春", "雨水", "惊蛰", "春分", "清明", "谷雨", \
            "立夏", "小满", "芒种", "夏至", "小暑", "大暑", \
            "立秋", "处暑", "白露", "秋分", "寒露", "霜降", \
            "立冬", "小雪", "大雪", "冬至", "小寒", "大寒"]
    
    for i in range(0,24):
        lst_jq[name_jq[i]]=lst[i]
        
    
        
    return lst_jq

def exp_jq(st,end):
    data={}
    for i in tqdm(range(st,end+1)):
        data[str(i)]=main_spider(i)
        time.sleep(1)
    t=json.dumps(data,ensure_ascii=False)
    
    fn=str(st)+'-'+str(end)+"年的节气.json"
    print(fn)
    with open(fn,'w') as f:
        f.write(t)
        
    print('done')



if __name__=="__main__":
    exp_jq(1800,2100)
    

In [88]:
import json
import re
from datetime import datetime
        
class ganzhi:
    def __init__(self):
        self.jqnames=["立春", "雨水", "惊蛰", "春分", "清明", "谷雨", \
                        "立夏", "小满", "芒种", "夏至", "小暑", "大暑", \
                        "立秋", "处暑", "白露", "秋分", "寒露", "霜降", \
                        "立冬", "小雪", "大雪", "冬至", "小寒", "大寒"]
        self.tg=['甲','乙','丙','丁','戊','己','庚','辛','壬','癸']
        self.dz=['子','丑','寅','卯','辰','巳','午','未','申','酉','戌','亥']
        
        with open('1800-2100jieqi.json','r') as f: 
            for line in f.readlines():
                self.dic = json.loads(line)
#         print('测试开始：',self.dic['2020'])
    
    def inputdate(self,y,m,d,h=-1,zishi=0):
        self.jqlist=self.dic[str(y)]
        ymd=str(y)+"-"+ str(m)+ '-' +str(d)
        jqdate=[]
        for i in self.jqnames:
            ptn=r'\d+年.+日'
            ptn_time=r'\d\d:\d\d:\d\d'
            ptn_lunar=r'农历.+'
            tmp_jq=re.search(ptn,self.jqlist[i])
            tmp_jq_time=re.search(ptn_time,self.jqlist[i])
            tmp_jq_lunar=re.search(ptn_lunar,self.jqlist[i])
            jqdate.append([i,tmp_jq[0],tmp_jq_time[0],tmp_jq_lunar[0]])
            
        for i in jqdate:
            i[1]=i[1].replace('年','-').replace('月','-')[0:-1]
            
        jqdate.insert(0,jqdate[23])
        jqdate.insert(0,jqdate[23])
        jqdate.pop()
        jqdate.pop()
        
        input_time=datetime.strptime(ymd, '%Y-%m-%d')
        jq_section=[]
        n=0
        for i in jqdate:
            list_time=datetime.strptime(i[1], '%Y-%m-%d')
            if list_time>input_time:
                jq_section.append(jqdate[n-1][0])
                jq_section.append(jqdate[n][0])
                break
            n+=1
        print(jq_section)
        
        
        for i in jqnames:
            
        

        
        


if __name__=='__main__':
    mybz=ganzhi()
    mybz.inputdate(2020,3,4)

['雨水', '惊蛰']


In [57]:
a='2020年2月4日 17:03:12 农历 正月(小)十一'
ptn_time=r'\d\d:\d\d:\d\d'
b=re.search(ptn_time,a)
print(b[0])

17:03:12
